In [ ]:
import numpy as np
import xarray as xr
import scipy.io as sio
import pandas as pd

import matplotlib.pyplot as plt

In [ ]:
rootdir = '/raid1/chen423/serdp/archive/GRL2020/'

plotdir = rootdir + 'plots/'

In [ ]:
def crt_refdata():    
    reffile = rootdir + 'data/common_ref/US_state.nc'
    wUS_mask_stack = xr.open_dataset(reffile).state_mask.values
    WRF_mask_wUS = wUS_mask_stack[0:3].sum(axis=0) + wUS_mask_stack[4]
    
    wUS_mask_by_region = wUS_mask_stack[0] + wUS_mask_stack[1] + 2*wUS_mask_stack[2] + 2*wUS_mask_stack[4]
    
    return 1-WRF_mask_wUS, wUS_mask_by_region

In [ ]:
mask_wUS, msk_wUS_2region = crt_refdata()

In [ ]:
ts_monthly = pd.period_range(start='2003-10', end='2015-09', freq='M')

reffile = rootdir + 'data/common_ref/latlon.nc'
wrf_lats = xr.open_dataset(reffile).XLAT_M.values
wrf_lons = xr.open_dataset(reffile).XLONG_M.values

slat = 32
elat = 48
lat_int = 1

n_lat_band = np.arange(slat, elat+0.0001, lat_int).shape[0]
lat_band_mask = np.zeros((450,450))

for i in np.arange(slat, elat+0.0001, lat_int):
    lat_band_mask[(wrf_lats>i)*(wrf_lats<=(i+lat_int))*(mask_wUS==0)] = i

In [ ]:
def compute_hovmollder_data(indata, slat=32, elat=48, int_lat=2):
    outdata = [indata[lat_band_mask==i].mean()/30/4 for i in np.arange(slat, elat+0.0001, int_lat)]
    return outdata

In [ ]:
def collect_ARTMIP_data(ARmethod):
    infile = rootdir + 'data/ARTMIP/ARTMIP.T1.MERRA2.%s.SERDP6km.3hr_daysum.200310-201509.monmean.nc' % ARmethod
    ARcount_in = xr.open_dataset(infile).ar_binary_tag.values/8
    
    # mean monthly frequency, maps
    months = [10, 11, 12, 1, 2, 3]
    outdata_map = np.zeros((6, 450, 450))
    for i in np.arange(6):
        outdata_map[i] = ARcount_in[ts_monthly.month==months[i]].mean(axis=0)
    
    # hovmoller
    outdata_hov = np.zeros((n_lat_band, 144))
    for i in np.arange(144):
        outdata_hov[:,i] = compute_hovmollder_data(ARcount_in[i]*30*4, int_lat=lat_int)
        
    # line plot
    nt = ARcount_in.shape[0]
    area_mean = np.zeros(nt)
    for i in np.arange(nt):
        area_mean[i] = (ARcount_in[i][mask_wUS==0]).mean()
    outdata_line = np.zeros(12)
    for i in np.arange(12):
        tmpdata = area_mean[ts_monthly.month==(i+1)]
        outdata_line[i] = tmpdata.mean()

    
    return outdata_map, outdata_hov, outdata_line

In [ ]:
ARnames = ['gershunov', 'goldenson', 'guan', 'lora', 'payne', 'pnnl1', 'pnnl2', 'rutz', 'tempest', 'walton']

In [ ]:
plotdata_map = np.zeros((10, 6, 450, 450))
plotdata_hov = np.zeros((10, 17, 144))
plotdata_line = np.zeros((10, 12))

for i in np.arange(10):
    print(ARnames[i])
    plotdata_map[i], plotdata_hov[i], plotdata_line[i] = collect_ARTMIP_data(ARnames[i])

## plots

In [ ]:
import cartopy
import cartopy.crs as ccrs

# for shapefile
from cartopy.io.shapereader import Reader
from cartopy.feature import ShapelyFeature

# NCL colormap
import matplotlib
import colormath, colormath.color_objects, colormath.color_conversions
from colormath.color_objects import sRGBColor
import urllib
import re


# ticks
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

In [ ]:
color_obj_dict = {'sRGB':colormath.color_objects.sRGBColor,
                  'HSV':colormath.color_objects.HSVColor,
                  'Lab':colormath.color_objects.LabColor,
                  'LCHuv':colormath.color_objects.LCHuvColor,
                  'LCHab':colormath.color_objects.LCHabColor,
                  'XYZ':colormath.color_objects.XYZColor}

def __rgb_to_array(rgb_color):
    r = np.minimum(1, round(rgb_color.rgb_r*10000)/10000)
    g = np.minimum(1, round(rgb_color.rgb_g*10000)/10000)
    b = np.minimum(1, round(rgb_color.rgb_b*10000)/10000)
    return r,g,b


def create_palette(start_rgb, end_rgb, n, colorspace):
    # convert start and end to a point in the given colorspace
    start = colormath.color_conversions.convert_color(start_rgb, colorspace).get_value_tuple()
    end = colormath.color_conversions.convert_color(end_rgb, colorspace).get_value_tuple()

    # create a set of n points along start to end
    points = list(zip(*[np.linspace(start[i], end[i], n) for i in range(3)]))

    # create a color for each point and convert back to rgb
    rgb_colors = [colormath.color_conversions.convert_color(colorspace(*point), sRGBColor) for point in points]

    # convert rgb colors to arrays
    return [__rgb_to_array(color) for color in  rgb_colors]


def __retrive_NCL_webcontent(cmapname):
    target_url = 'https://www.ncl.ucar.edu/Document/Graphics/ColorTables/Files/%s.rgb' % cmapname
    request = urllib.request.urlopen(target_url)
    return request


def __collect_discrete_NCL_cmap(cmapname):
    rawdata = __retrive_NCL_webcontent(cmapname)
    
    cmap_color_list = list()
    
    color_section_sig = 0
    
    for line in rawdata:
        
        line_decode = line.decode('utf-8')
        info = re.split('\s+', line_decode.replace('\n','').replace('^\s+',''))
        
        if color_section_sig==1:
            if info[0]=='' and len(info)>=3:
                if np.maximum(np.maximum(float(info[1]), float(info[2])), float(info[3]))>1:
                    cmap_color_list.append((float(info[1])/255, float(info[2])/255, float(info[3])/255))
                else:
                    cmap_color_list.append((float(info[1]), float(info[2]), float(info[3])))
            if len(info)==3:
                if ';' in info[0] or '#' in info[0]:
                    whatisthis = 's'
                else:
                    if np.maximum(np.maximum(float(info[0]), float(info[1])), float(info[2]))>1:
                        cmap_color_list.append((float(info[0])/255, float(info[1])/255, float(info[2])/255))
                    else:
                        cmap_color_list.append((float(info[0]), float(info[1]), float(info[2])))
        
        if 'ncolors' in str(info[0]):
            color_section_sig = 1  # meaning now we are at color lines (or "r g b" line)

    return cmap_color_list


def __cmap_refinement(raw_cmap_rgb, n_interpolate=10, workspace=color_obj_dict['sRGB']):
    # workspace:  choose which color space the refinement is conducted.
    #             refer to https://stackoverflow.com/questions/55032648/given-a-start-color-and-a-middle-color-how-to-get-the-remaining-colors-python
    
    n_in = len(raw_cmap_rgb)

    new_array = list()

    for i in np.arange(n_in-1):
        out_colors = create_palette(sRGBColor(*raw_cmap_rgb[i], is_upscaled=False), sRGBColor(*raw_cmap_rgb[i+1], is_upscaled=False), n_interpolate+1, workspace)
        for j in np.arange(len(out_colors)-1):
            new_array.append(out_colors[j])
            
    return new_array


def generate_NCL_cmap(cmapname, cont_opt=False, cont_param_n=10, cont_param_ws='sRGB',
                      white_first=False, white_ext=False, reverse_cmap=False):
    # description:
    #     cmapname:      taken as shown on the NCL website
    #     cont_opt:      to convert the discreate colormap to continuous colormap
    #     cont_param_n:  how many "intermediate" colors to be inserted to the nearby discreate colors
    #     cont_param_ws: color space to conduct interploation. Default to "sRGB", which should work for most cases
    #     white_first:   whether to set the first color as white. May be useful if the minimum does not mean anything
    
    cmap_discrete_raw = __collect_discrete_NCL_cmap(cmapname)
    
    if reverse_cmap==True:
        cmap_discrete_raw.reverse()
    
    if white_first==True:
        if white_ext==True:
            cmap_discrete = list()
            cmap_discrete.append((1,1,1))
            for i in np.arange(len(cmap_discrete_raw)):
                cmap_discrete.append(cmap_discrete_raw[int(i)])
        else:
            cmap_discrete = cmap_discrete_raw.copy()
        cmap_discrete[0] = (1,1,1)
    else:
        cmap_discrete = cmap_discrete_raw
    
    if cont_opt==False:
        out_cmap = cmap_discrete
        
    if cont_opt==True:
        out_cmap = __cmap_refinement(cmap_discrete, n_interpolate=cont_param_n, workspace=color_obj_dict[cont_param_ws])
        
    return matplotlib.colors.ListedColormap(out_cmap)


def crt_cbar_labels(vmax, n_interval, mode='diff', decimal_flag=0, perc_flag=False, vmin=0):
    # crt_cbar_labels:  create the colorbar label lists
    #   mode:  choose between "diff" and "0ton". "diff" means setting the colorbar as -vmax to vmax, "0ton" 
    #          means setting the colorbar as 0 to vmax
    #   n_interval:  how many segments are there? See example below.
    #   decimal_flag:  control the text format. Default to 0.
    #
    # Example:
    #      > crt_cbar_labels(80, 4, mode='diff', decimal_flag=0)
    #      >  ['-80', '-40', '0', '40', '80']
    #      > crt_cbar_labels(80, 4, mode='0ton', decimal_flag=1)
    #      >  ['0.0', '20.0', '40.0', '60.0', '80.0']
    
    if perc_flag==True:
        format_string = '%%.%df%%%%' % (decimal_flag)
    else:
        format_string = '%%.%df' % (decimal_flag)
    #print(format_string)
    outdata = []
    
    if mode=='diff':
        n_interval = n_interval/2
        for i in np.arange(-1*n_interval, n_interval+0.000001, 1):
            outdata.append(format_string%(vmax*i/n_interval))
    if mode=='0ton':
        for i in np.arange(0, n_interval+0.000001, 1):
            outdata.append(format_string%(vmax*i/n_interval))
    if mode=='minmax':
        for i in np.arange(0, n_interval+0.000001, 1):
            outdata.append(format_string%(vmin + (vmax-vmin)*i/n_interval))
        
    return outdata

In [ ]:
cmap_map = generate_NCL_cmap('perc2_9lev', cont_opt=True)
cmap_hov = generate_NCL_cmap('precip4_11lev', cont_opt=True)

mask_to_use = mask_wUS

vmax_map = 0.24
vmax_hov = 0.25


fig1 = plt.figure(figsize=(7,7))

ax1 = plt.subplot2grid((30,10), (0,0), rowspan=8, colspan=9)
ax2 = plt.subplot2grid((30,10), (9,0), rowspan=8, colspan=9)
ax3 = plt.subplot2grid((30,10), (22,0), rowspan=8, colspan=9)

ax1.pcolormesh(np.arange(144), np.arange(slat, elat+0.000001, lat_int), plotdata_hov[0], cmap=cmap_hov, vmin=0, vmax=vmax_hov)
ax2.pcolormesh(np.arange(144), np.arange(slat, elat+0.000001, lat_int), plotdata_hov.mean(axis=0), cmap=cmap_hov, vmin=0, vmax=vmax_hov)

ax3.plot(np.arange(1,13), plotdata_line[1].T, color='royalblue', label='ARTMIP-catalogues')
ax3.plot(np.arange(1,13), plotdata_line[2:10].T, color='royalblue')
ax3.plot(np.arange(1,13), plotdata_line[0], linestyle='--', linewidth=3, color='orchid', label='ARTMIP-Gershunov')
ax3.set_xlim([0.8, 12.2])
ax3.set_xticks(np.arange(1,13))
ax3.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
ax3.set_xlabel('Month')
ax3.set_ylabel('mean fraction')
ax3.set_yticks(np.arange(0, 0.121, 0.04))
ax3.legend(loc='upper left', ncol=2, frameon=False)

cbar_ax1 = fig1.add_axes([0.84, 0.45, 0.01, 0.4])
cb1 = matplotlib.colorbar.ColorbarBase(cbar_ax1, cmap=cmap_hov,
                                       ticks=np.arange(0, 1.00001, 1/5), orientation='vertical')
cb1.set_ticklabels(crt_cbar_labels(vmax_hov, 5, mode='0ton', decimal_flag=2))
cbar_ax1.tick_params(labelsize=9)
cbar_ax1.text(1, -0.1, '(frac)', size=10)

for axis in [ax1, ax2]:
    axis.set_ylabel('latitude')
    axis.set_xticks(np.arange(3, 144, 12))
    axis.set_xticklabels([])
    axis.xaxis.set_minor_locator(MultipleLocator(1))
    axis.set_yticks(np.arange(35, 46, 5))
    axis.yaxis.set_minor_locator(MultipleLocator(2.5))

ax1.set_title('AR frequency over western U.S. (2003-2015)', fontsize=12)
ax2.set_xticklabels(np.arange(2004, 2016))
ax2.set_xlabel('Year')

ax1.text(140, 33, '(a) ARTMIP-Gershunov', ha='right', va='bottom', backgroundcolor='white', fontsize=12)
ax2.text(140, 33, '(b) ARTMIP-mean', ha='right', va='bottom', backgroundcolor='white', fontsize=12)

ax3.set_title('(c) Monthly cycle of AR frequency (2003-2015)', fontsize=12)

#figname = plotdir + 'fig_S8.ARTMIP_distribution.png'
#fig1.savefig(figname, dpi=600)

plt.show()
plt.close()
del(fig1)